# Imports

In [1]:
import os

from dotenv import load_dotenv
import torch
from torch.utils.data import DataLoader
import lightning as pl
from lightning.pytorch.loggers import MLFlowLogger
import mlflow.pytorch

from src.datasets import SurveillanceAnomalyDataset
from src.hstgcnn import HSTGCNN


load_dotenv()

False

# Data loaders

In [2]:
dataset_path = "./data/processed/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train"
BATCH_SIZE = 64
num_workers = 4

# Create dataset and dataloaders
dataset = SurveillanceAnomalyDataset(dataset_path)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    persistent_workers=True,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    persistent_workers=True,
)

# Training loop

In [3]:
DEVICE = "cuda"
EPOCHS = 20
learning_rate = 1e-3

experiment_name = "HSTGCNN_UCSD_Ped1"
model_version = "1.0.0"
run_name = f"HSTGCNN@{model_version}"

model_definition_path = "./src/hstgcnn.py"

In [4]:
mlflow.set_experiment(experiment_name)
mlf_logger = MLFlowLogger(
    tracking_uri=os.getenv("MLFLOW_TRACKING_URI"),
    experiment_name=experiment_name,
    run_name=run_name,
    log_model=False,
)
# Initialize trainer
trainer = pl.Trainer(
    max_epochs=EPOCHS,
    accelerator=DEVICE,
    log_every_n_steps=1,
    logger=mlf_logger,
    callbacks=[
        pl.pytorch.callbacks.EarlyStopping(monitor='val_loss', patience=3, mode='min'),
        pl.pytorch.callbacks.ModelCheckpoint(dirpath='./checkpoints', filename='hstgcnn-epoch_{epoch:02d}',
                                     monitor='val_loss', mode='min', save_top_k=1,
                                     save_last=False, auto_insert_metric_name=False),
    ],
)

model = HSTGCNN(oa_weights=[0.2, 0.5, 0.3], lr=learning_rate).to(DEVICE)
# Log hyperparameters to MLflow
mlf_logger.log_hyperparams({
    'model_name': model.__class__.__name__,
    'learning_rate': learning_rate,
    'batch_size': BATCH_SIZE,
    'max_epochs': EPOCHS,
})
# Train the model
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
# Log artifacts
best_model_path = trainer.checkpoint_callback.best_model_path
if best_model_path:
    mlf_logger.experiment.log_artifact(mlf_logger.run_id, best_model_path, "models")
    mlf_logger.experiment.log_artifact(mlf_logger.run_id, model_definition_path, "models")

2025/06/03 13:24:06 INFO mlflow.tracking.fluent: Experiment with name 'HSTGCNN_UCSD_Ped1' does not exist. Creating a new experiment.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type          | Params | Mode 
------------------------------------------------------------
0 | high_level_stgcnn | STGCNN        | 97     | train
1 | low_level_stgcnn  | STGCNN        | 193    | train
2 | feature_fusion    | FeatureFusion | 12.4 K | train
3 | ffp               | FFP           | 199 K  | train
4 | oa                | OA            | 3      | train
5 | loss_fn           | MSELoss       | 0      | train
------------------------------------------------------------
211 K     Trainable params
0         Non-trainable params
211 K     Tota

Epoch 0: 100%|█████████████| 83/83 [23:07<00:00,  0.06it/s, v_num=35ff, train_loss=1.94e+3]
Validation: |                                                        | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|█████████████████████████████| 21/21 [04:19<00:00,  0.08it/s]
Epoch 1: 100%|█| 83/83 [21:42<00:00,  0.06it/s, v_num=35ff, train_loss=774.0, val_loss=2.47
Validation: |                                                        | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|█████████████████████████████| 21/21 [04:13<00:00,  0.08it/s]
Epoch 2: 100%|█| 83/83 [21:53<00:00,  0.06it/s, v_num=35ff, train_loss=1.16e+3, val_loss=85
Validation: |                                                        | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|█████████████████████████████| 21/21 [04:24<00:00,  0.08it/s]
Epoch 3: 100%|█| 83/83 [21:52<00:00,  0.06it/s, v_num=35ff, train_loss=766.0, val_loss=864.
Validation: |                                                        | 0/? [00:0